# DINOv2 Exploration

This notebook is used for exploring the DINOv2 model, including loading the model, running predictions, and visualizing results.

In [1]:
from src.models.dinov2_model import Dinov2Model
from src.utils.visualization import display_image
import matplotlib.pyplot as plt

# Initialize the DINOv2 model
dinov2_model = Dinov2Model()
dinov2_model.load_model()


In [2]:
# Preprocess an input image
input_image = 'path/to/your/image.jpg'  # Replace with your image path
preprocessed_image = dinov2_model.preprocess_input(input_image)


In [3]:
# Run prediction
predictions = dinov2_model.predict(preprocessed_image)
print(predictions)


In [4]:
# Visualize the results
plt.figure(figsize=(10, 5))
display_image(input_image)
plt.title('Input Image')
plt.axis('off')
plt.show()